# 뷰 템플릿 적용하기
1. ejs 뷰 템플릿 적용하기
2. pug 뷰 템플릿 적용하기

## ejs 뷰 템플릿 사용하기

#### 뷰 템플릿이란?
* 응답 웹문서를 미리 뷰 템플레이트로 만들어두고 응답 웹문서를 만들 때 데이터와 템플레이트를 결합하여 만드는 방식 사용

```
app.set('views', __dirname + '/views');
app.set('view engine', 'ejs');
```

In [4]:
%%writefile ./src/views/login_success.ejs
<!DOCTYPE html>
<html>
    <head>
        <meta charset="UTF-8">
        <title>로그인 성공 페이지</title>
    </head>
    <body>
        <h1>로그인 성공</h1>
        <div><p>사용자 아이디 : <%= userid %> </p></div>
        <div><p>사용자 이름 : <%= username %> </p></div>
        <br><br><a href='/public/login.html'>다시 로그인하기</a>
    </body>
</html>

Writing ./src/views/login_success.ejs


In [5]:
%%writefile ./src/routes/user.js
var login = function(req,res){
    console.log('/process/login 라우팅 함수 호출됨.');
        
    var paramId = req.body.id || req.query.id;
    var paramPassword = req.body.password || req.query.password;
    console.log('요청 파라미터 : ' + paramId + ', ' + paramPassword);    
    
    var database = req.app.get('database');
    if (database) {
        authUser(database, paramId, paramPassword, function(err, docs) {
            if(err){
                console.log('에러 발생.');
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>에러 발생</h1>');
                res.end();
                return;
            }
                    
            if(docs){
                console.dir(docs);
                
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                    
                var context = {
                    userid: paramId,
                    username: docs[0].name
                }
                req.app.render('login_success', context, function(err, html){
                    if(err){
                        console.log('뷰 렌더링 중 에러 발생 : ' + err.stack);
                        console.log('에러 발생.');
                                
                        res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                        res.write('<h1>뷰 렌더링 중 에러 발생</h1>');
                        res.write('<br><p>' + err.stack + '</p>');
                        res.end();
                        return;
                    }
                    res.end(html);
                })
                
            }else{
                console.log('에러 발생.');
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 데이터 조회 안됨.</h1>');
                res.end();
                return;
            }
        })    
    }else {
        console.log('에러 발생.');
        res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
        res.write('<h1>데이터베이스 연결 안됨.</h1>');
        res.end();
        return;    
    }
}

var adduser = function(req,res){
    console.log('/process/adduser 라우팅 함수 호출됨.');
    var paramId=req.body.id || req.query.id;
    var paramPassword = req.body.password || req.query.password;
    var paramName = req.body.name || req.query.name;
        
    var database = req.app.get('database');
    console.log('요청 파라미터 : ' + paramId + ', ' + paramPassword + ', '+paramName);
    if(database) {
        addUser(database, paramId, paramPassword, paramName, function(err, result){
            if(err) throw err;
            if(result){
                console.dir(result);
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 추가 성공</h1>');
                res.write('<div><p>사용자 : ' + paramName + '</p></div>');
                res.write('<br><br><a href="/public/login.html">다시 로그인하기</a>');
                res.end();
            } else {
                console.log('에러 발생');
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 추가 안됨</h1>');
                res.end();    
            }
        });
    }else {
        console.log('에러 발생.');
        res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
        res.write('<h1>데이터베이스 연결 안됨.</h1>');
        res.end();
        return;  
    }   
}

var listuser = function(req,res){
    console.log('/process/listuser 라우팅 함수 호출됨.');
    
    var database = req.app.get('database');
    if(database) {
        database.UserModel.findAll(function(err, results){
            if(err){
                console.log('에러 발생.');
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 데이터 조회 안됨.</h1>');
                res.end();
                return;
            }
            if(results){
                console.dir(results);
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write("<h3>사용자 리스트</h3>");
                res.write("<div><ul>");
                for(var i=0; i<results.length; i++){
                    var curId = results[i]._doc.id;
                    var curName = results[i]._doc.name;
                    res.write("    <li>#" + i + "-> " + curId + ", " + curName +
                             "</li>");
                }
                res.write("</ul></div>");
                res.end();
            }else {
                console.log('에러 발생.');
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>조회된 사용자 없음.</h1>');
                res.end();
                return;
            }
        })
    }else{
        console.log('에러 발생.');
        res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
        res.write('<h1>데이터베이스 연결안됨.</h1>');
        res.end();
        return;
    }
}

var authUser = function(database, id, password, callback) {
    console.log('authUser 호출됨 : ' + id + ', ' + password);
    
    database.UserModel.findById(id, function(err, result) {
        if(err) {
            callback(err, null);
            return;
        }
        console.log('아이디 %s로 검색됨.');
        if(result.length > 0){
            var user = new UserModel({id:id})
            var authenticated = user.authenticate(password, result[0],_doc.salt, result[0]._doc.hashed_password);
                
            if(authenticated) {
                console.log('비밀번호 일치함.');
                callback(null, result);
            } else {
                console.log('비밀번호 일치하지 않음.');
                callback(null, null);
            }
        }else{
            console.log('아이디 일치하는 사용자 없음.');
            callback(null, null);
        }
    });
    
}

var addUser = function(database, id, password, name, callback) {
    console.log('addUuser 호출됨 : ' + id + ', ' + password + ', '+ name);
    
    var user = new database.UserModel({"id":id,"password":password,"name":name});
    
    user.save(function(err) {
        if(err) {
            callback(err, null);
            return;
        }
        console.log('사용자 데이터 추가함.');
        callback(null, user);
    });
}

module.exports.login = login;
module.exports.adduser = adduser;
module.exports.listuser = listuser;

Overwriting ./src/routes/user.js


```                    
var context = {
    //userid: paramId,
    //username: docs[0].name
    results: results
}
req.app.render('login_success', context, function(err, html){
    if(err){
        console.log('뷰 렌더링 중 에러 발생 : ' + err.stack);
        console.log('에러 발생.');

        res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
        res.write('<h1>뷰 렌더링 중 에러 발생</h1>');
        res.write('<br><p>' + err.stack + '</p>');
        res.end();
        return;
    }
    res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
    res.end(html);
})
```

```
<% 
for (var i=0; i<results.length; i++){
    //blahblah
    var curId = results[i]._doc.id;
    var curName = results[i]._doc.name; 
%>
    <li># <%= i %> - 아이디 : <%=curId %>, 이름 : <%=curName%></li>
<%
}
%>
```

## pug 뷰 템플레이트 사용하기
* 예전 이름은 jade 였음.
* 최대한 간단한 형태로 입력하므로 공백과 들여쓰기를 기준으로 태그의 구조가 결정됨
* 들여쓰기를 잘못하면 문제 발생함
* 모듈 설치하고 뷰 엔진 설정 필요

```
app.js파일에 아래와 같이 추가해야함

app.set('views', __dirname + '/views');
app.set('view engine', 'pug');
```

In [8]:
%%writefile ./src/test.js
var test1 = function(req, res){
    console.log('test 모듈 안에 있는 test1 호출됨.');
    
    var context = {};
    req.app.render('test1_success', context, function(err, html) {
            console.log('HTML -> ' + html);
            
            res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
            res.end(html);
        })
}
module.exports.test1 = test1;

Writing ./src/test.js


In [10]:
%%writefile ./src/config.js
module.exports = {
    server_port: 3000,
    db_url: 'mongodb://localhost:27017/local',
    db_schemas: [
        {file:'./user_schema', collection:'users3', schemaName:'UserSchema', modelName:'UserModel'}
    ],
    route_info: [
        //===== User =====//
        {file:'./user', path:'/process/login', method:'login', type:'post'} // user.login 
        ,{file:'./user', path:'/process/adduser', method:'adduser', type:'post'} // user.adduser 
        ,{file:'./user', path:'/process/listuser', method:'listuser', type:'post'} // user.listuser 
        ,{file:'./test', path:'/process/test1', method:'test1', type:'get'}
    ]
}

Overwriting ./src/config.js


In [7]:
%%writefile ./src/views/test1_success.pug
doctype html

html
  head
    title "테스트 성공"
  body
    block content
      #container
        p "조회에 성공했습니다."

Writing ./src/views/test1_success.pug


D:\폴더\학교\졸프\evote-project\WebServer\Study\src>npm install pug-cli --save

D:\폴더\학교\졸프\evote-project\WebServer\Study\src>pug test1_success.pug --pretty

## ejs 가 유지관리가 더 편하다.
## pug는 심플하다